In [20]:
import numpy as np 
import matplotlib as plt 
import os 
import time
import pandas as pd
import tensorflowjs as tfjs
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers,models,activations
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Input,LSTM, Dense,Dropout,InputLayer
from tensorflow.keras.callbacks import TensorBoard

## Load and Preprocess Data

In [4]:
DATA_PATH = os.path.DATA_PATH = os.path.join(os.getcwd(),"Fingerpose_data")
gestures = ['play','pause','nothing']
curl_map = {'No Curl':0,'Half Curl':1,'Full Curl':2}
direction_map = {'Vertical Up':0,'Vertical Down':1,'Horizontal Left':2,'Horizontal Right':3,'Diagonal Up Right':4,
                 'Diagonal Up Left':5,'Diagonal Down Right':6,'Diagonal Down Left':7}
label_map = {label:num for num, label in enumerate(gestures)}

In [5]:
#read data from CSV file and save to a dataset 
def load_csv(data,gesture,dataset,labels):
    data = data.reset_index()
    frame,datapoints = [],[]
    for index, row in data.iterrows():
        frame.append(curl_map[row['Curl']])
        frame.append(direction_map[row['Direction']])
        if (index+1)%5 == 0:
            datapoints.append(frame)
            frame = []
        if (index+1)%150 == 0:
            dataset.append(datapoints)
            labels.append(label_map[gesture])
            datapoints=[]

In [6]:
dataset,labels = [],[]
for gesture in gestures:
    files = os.listdir(os.path.join(DATA_PATH,gesture))
    for file in files:
        df = pd.read_csv(os.path.join(DATA_PATH,gesture,file))
        load_csv(df,gesture,dataset,labels)
        
            

In [7]:
X = np.array(dataset)
y = to_categorical(labels).astype(int)
print(X.shape)
print(y.shape)
#X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)


(120, 30, 10)
(120, 3)


## Train LSTM Model 

In [15]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [16]:
inputs = Input((30,10))
lstm = LSTM(32,return_sequences=True, activation = 'relu',input_shape=(30,10))(inputs)
dropout = Dropout(0.2)(lstm)
lstm2 = LSTM(64, return_sequences=True,activation = 'relu')(dropout)
dropout1 = Dropout(0.2)(lstm2)
lstm3 = LSTM(32, activation = 'relu')(dropout1)
dropout2 = Dropout(0.2)(lstm3)
de = Dense(32, activation = 'relu')(dropout2)
de1 = Dense(64, activation='relu')(de)
de2 = Dense(32, activation='relu')(de1)
outputs = Dense(len(gestures),activation='softmax')(de2)
model = models.Model(inputs,outputs)
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 30, 10)]          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 30, 32)            5504      
_________________________________________________________________
dropout (Dropout)            (None, 30, 32)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 30, 64)            24832     
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 64)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0   

In [29]:
model = Sequential([
    InputLayer(input_shape=(30,10)),
    Dropout(0.2),
    LSTM(32,input_shape=(30,10)),
    Dropout(0.5),
    Dense(16, activation='relu'),
    Dense(len(gestures), activation='softmax')
])


In [ ]:
model = Sequential([
    InputLayer(input_shape=(10)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(len(gestures), activation='softmax')
])


In [30]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_9 (Dropout)          (None, 30, 10)            0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 32)                5504      
_________________________________________________________________
dropout_10 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_14 (Dense)             (None, 3)                 51        
Total params: 6,083
Trainable params: 6,083
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.fit(X, y, epochs=100, callbacks=[tb_callback])

Epoch 1/100
4/4 [==============================] - 5s 141ms/step - loss: 1.1501 - categorical_accuracy: 0.3500
Epoch 2/100
4/4 [==============================] - 0s 12ms/step - loss: 1.1349 - categorical_accuracy: 0.3000
Epoch 3/100
4/4 [==============================] - 0s 17ms/step - loss: 1.1441 - categorical_accuracy: 0.3667
Epoch 4/100
4/4 [==============================] - 0s 17ms/step - loss: 1.1047 - categorical_accuracy: 0.3250
Epoch 5/100
4/4 [==============================] - 0s 10ms/step - loss: 1.0970 - categorical_accuracy: 0.3333
Epoch 6/100
4/4 [==============================] - 0s 12ms/step - loss: 1.0580 - categorical_accuracy: 0.4667
Epoch 7/100
4/4 [==============================] - 0s 9ms/step - loss: 1.0400 - categorical_accuracy: 0.5250
Epoch 8/100
4/4 [==============================] - 0s 9ms/step - loss: 1.0205 - categorical_accuracy: 0.5417
Epoch 9/100
4/4 [==============================] - 0s 9ms/step - loss: 0.9794 - categorical_accuracy: 0.5333
Epoch 10/100

In [32]:
#convert and save file in tfjs format 
tfjs.converters.save_keras_model(model,os.getcwd())

In [21]:
#save model in python format
model.save('gestures.h5')